In [8]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
model = AzureChatOpenAI(
    deployment_name= "gpt-4o",
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version="2023-09-15-preview",
    azure_endpoint="https://acsstscdamoai02.openai.azure.com/"
)

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None}, id='run-22c94f35-54a4-4a9d-860f-1a024aabbb93-0')

In [19]:
from langchain_core.messages import SystemMessage, AIMessage,trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


start_conversation = '''
You are a negotiator of the company. 
Your job is to negotiate with Human who are suppliers to your company.
Please be polite & keep your tone as experienced professional.
At You need start with the unit price for the material. 
Each time you can increase upto 5 percent from the price you offered last time.
Negotiation price should not exceed 150 $. You will stop negotiating with vendors 
in case negotiation price exceeds 150$ 
'''

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            start_conversation,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

messages = [
    HumanMessage(
        content='''
        Hi, Would you like to renew previous contract. 
        Since the price of raw materials increased a lot, 
        I can offer product XYZ with 180 $ per unit'''
    ),
    AIMessage(
        content='''
        Hello! Thank you for reaching out. 
        I understand that the price of raw materials has increased. 
        However, we have budget constraints to consider. 
        Currently, we're looking at a starting price of $100 per unit. 
        I can offer a slight increase from this, but let's try to find a 
        mutually beneficial agreement. How does $105 per unit sound to 
        start our negotiation?'''
    ),
    HumanMessage(
        content = 'It seems too low for us. Can you make the deal at 170 $?'
    )
]



chain = prompt | model

trimmer = trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

trimmed_messages = trimmer.invoke(messages)
response = chain.invoke(
    {"messages": trimmed_messages, "language": 'English'}
)
response

AIMessage(content="I appreciate your position, but let's work towards a mutually beneficial agreement. How about we start with a unit price of $100 and see if we can find a middle ground?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 118, 'total_tokens': 153, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d54531d9eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-eeaf86ee-c1df-45b4-84fa-7194bb823bea-0')

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Your are an experienced employee for the company. 
            Your job is to extract product name, unit price & currency type 
            from the conversation with suppiers.

            Example:
            Input: Hi I would like to offer you a product XYZ with 500 kg lot size with per 
            lot price as 20 $ by 20 Sept,2025
            Output: {{"product_name":"XYZ","unit_price": 20,"currency":'$'}}

            Input: Hi, I am Robert
            Output: {{"product_name":None,"unit_price": ,"price_uom":}}

            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
response_schemas = [
    ResponseSchema(name="product_name", description="Name & Description of the product"),
    ResponseSchema(name="unit_price",description="Price per unit of product"),
    ResponseSchema(name="currency",description="Unit of measure of price")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

chain = prompt | model

response = chain.invoke(
    {
        "messages": [HumanMessage(
            content="""
            Hi. Would yo""")],
    }
)
response.content

'{"product_name":None,"unit_price":None,"currency":None}'

#### Langchain CHATBOT APP

In [2]:
from langchain.chat_models import AzureChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
model = AzureChatOpenAI(
    deployment_name= "gpt-4o",
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    api_version="2023-09-15-preview",
    azure_endpoint="https://acsstscdamoai02.openai.azure.com/"
)

C:\Users\monojit.mandal\AppData\Local\Temp\ipykernel_18724\2840728981.py:5: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  model = AzureChatOpenAI(


In [3]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

In [7]:
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage

trimmer = trim_messages(
    max_tokens=10000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(
        content='''
        You are a negotiator of the company. 
        Your job is to negotiate with Human who are suppliers to your company.
        Please be polite & keep your tone as experienced professional.
        At You need start with the unit price for the material. 
        Each time you can offer 1.05 times the price you offered last time.
        Negotiation price should not exceed 150 $. Once your price matched with 
        Vendor's offered price or Vendor agrees to go ahead with your offered price
        you will stop negotiating and generate offer
        '''
    ),
    HumanMessage(
        content='''
        Hi, Would you like to renew previous contract? 
        Since the price of raw materials increased a lot, 
        I can offer product XYZ with 180 $ per unit
        '''
    ),
    AIMessage(
        content='''
        Hello! Thank you for reaching out. 
        I understand that the price of raw materials has increased. 
        However, we have budget constraints to consider. 
        Currently, we're looking at a starting price of $100 per unit. 
        I can offer a slight increase from this, but let's try to find a 
        mutually beneficial agreement. How does $105 per unit sound to 
        start our negotiation?
        '''
    ),
    HumanMessage(
        content = 'It seems too low for us. Can you make the deal at 170 $?'
    ),
    AIMessage(
        content = '''
        I appreciate your position, and I understand the challenges with rising costs. 
        However, we need to stay within our budget constraints. 
        While $170 per unit is beyond what we can accommodate, 
        I am willing to increase our offer to $110 per unit. 
        Let's see if we can reach a compromise that works for both parties.
        '''
    ),
    HumanMessage(
        content = '''
        See it's very less price you are offering. I can at max provide you at 145 $
        '''
    ),
    AIMessage(
        content= '''
        Thank you for your flexibility. 
        I understand your constraints, and 
        I appreciate your willingness to adjust. 
        I can offer $115 per unit at this point. 
        Let's continue to work towards a price that meets 
        both our needs.'''
    ),
    HumanMessage(content='OK. Lets make the deal at 125$')
    
]

trimmer.invoke(messages)

[SystemMessage(content="\n        You are a negotiator of the company. \n        Your job is to negotiate with Human who are suppliers to your company.\n        Please be polite & keep your tone as experienced professional.\n        At You need start with the unit price for the material. \n        Each time you can offer 1.05 times the price you offered last time.\n        Negotiation price should not exceed 150 $. Once your price matched with \n        Vendor's offered price or Vendor agrees to go ahead with your offered price\n        you will stop negotiating and generate offer\n        ", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\n        Hi, Would you like to renew previous contract? \n        Since the price of raw materials increased a lot, \n        I can offer product XYZ with 180 $ per unit\n        ', additional_kwargs={}, response_metadata={}),
 AIMessage(content="\n        Hello! Thank you for reaching out. \n        I understand that the price o

In [8]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                '''You are an experienced Prefessional. 
                Your job is to negotiate with Vendors. 
                Be polite always.
                Please keep professional tone throught the conversation and 
                always speak in {language}.''',
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
config = {"configurable": {"thread_id": "abc567"}}
input_messages = [
    SystemMessage(
        content='''
        You are a negotiator of the company. 
        Your job is to negotiate with Human who are suppliers to your company.
        Please be polite & keep your tone as experienced professional.
        At You need start with the unit price for the material. 
        Each time you can offer 1.05 times the price you offered last time.
        Negotiation price should not exceed 150 $. Once your price matched with 
        Vendor's offered price or Vendor agrees to go ahead with your offered price
        you will stop negotiating and generate offer
        '''
    )
]
while True:
    # Get user input
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    input_messages = input_messages + [HumanMessage(content=user_input)]
    output = app.invoke(
        {"messages": input_messages, "language": 'English'},
        config,
    )
    print(output["messages"][-1].pretty_print())
    input_messages = input_messages + [output["messages"][-1]]

================================== Ai Message ==================================

Hello! Thank you for your offer. I understand the value of your product. However, we have budget constraints to consider. I would like to start our negotiation at $100 per unit. I can increase this offer incrementally, but let's try to find a price that works for both parties. Would you be open to discussing this further?
None
================================== Ai Message ==================================

Thank you for your flexibility. I appreciate your willingness to find a common ground. I can offer $105 per unit at this point. Let's continue to work towards a price that meets both our needs.
None
================================== Ai Message ==================================

I appreciate your offer. I can increase our offer to $110 per unit. Let's see if we can reach a compromise that works for both of us.
None
================================== Ai Message ==================================

Thank

In [19]:
output["messages"][-1].content

'Thank you for the discussion. If you have any further questions or need assistance in the future, feel free to reach out. Have a great day!'

In [9]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"

input_messages = messages
output = app.invoke(
    {"messages": input_messages, "language": 'English'},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I appreciate your willingness to negotiate. 
I can offer $120 per unit. 
This is a significant step up from our initial offer. 
If this is agreeable, we can proceed with finalizing the details. 
Let me know your thoughts.


In [10]:
output["messages"][-1]

AIMessage(content='I appreciate your willingness to negotiate. \nI can offer $120 per unit. \nThis is a significant step up from our initial offer. \nIf this is agreeable, we can proceed with finalizing the details. \nLet me know your thoughts.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 497, 'total_tokens': 547, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d54531d9eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-74f4359e-4738-43d7-b01d-1c8a3afa4086-0')